In [1]:
import torch
from torch import nn
from torch.nn import functional
from attention import SelfAttention, CrossAttention
from decoder import VAE_AttentionBlock, VAE_ResidualBlock

ImportError: cannot import name 'SelfAttention' from 'attention' (c:\ProgramData\anaconda3\Lib\site-packages\attention\__init__.py)

In [ ]:
#https://ezyang.github.io/convolution-visualizer/index.html

In [ ]:
class VAE_Encoder(nn.Sequential): # reducing the size of the image but increasing the number of channels and the number of features
    def __init__(self):
        super().__init__(
            # (Batch_Size, Channel, Height, Width) -> (Batch_Size, 128, Height, Width)
            nn.Conv2d(3, 128, kernel_size=3, padding=1),
            

In [2]:
class TimeEmbedding(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.linear_1 = nn.Linear(n_embd, 4 * n_embd) # 320 -> 1280 
        self.linear_2 = nn.Linear(4 * n_embd, 4 * n_embd) # 1280 -> 1280

    def forward(self, x):
        # x: (1, 320)

        # (1, 320) -> (1, 1280)
        x = self.linear_1(x)
        
        # (1, 1280) -> (1, 1280)
        # activation function: silu
        x = functional.silu(x) 
        
        # (1, 1280) -> (1, 1280)
        x = self.linear_2(x)

        return x

In [3]:
class Upsample(nn.Module):
    def __init__(self, channels):
        super().__init__()
        #This line initializes a 2D convolutional layer with the same number of input and output channels, a kernel size of 3, and padding of 1.
        self.conv = nn.Conv2d(channels, channels, kernel_size=3, padding=1) # (Batch_Size, Features, Height, Width) -> (Batch_Size, Features, Height, Width) why paddding = 1  2*1+1 = 3 w
    
    def forward(self, x):
        # (Batch_Size, Features, Height, Width) -> (Batch_Size, Features, Height * 2, Width * 2)
        x = functional.interpolate(x, scale_factor=2, mode='nearest')  #upscaled by a factor of 2 using nearest neighbor interpolation, effectively doubles the height and width of the input feature maps, increasing their resolution.
        return self.conv(x)

In [ ]:
class UNET_OutputLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.groupnorm = nn.GroupNorm(32, in_channels) #https://pytorch.org/docs/stable/generated/torch.nn.GroupNorm.html
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
    
    def forward(self, x):
        # x: (Batch_Size, 320, Height / 8, Width / 8)

        # (Batch_Size, 320, Height / 8, Width / 8) -> (Batch_Size, 320, Height / 8, Width / 8)
        x = self.groupnorm(x)
        
        # (Batch_Size, 320, Height / 8, Width / 8) -> (Batch_Size, 320, Height / 8, Width / 8)
        x = functional.silu(x)
        
        # (Batch_Size, 320, Height / 8, Width / 8) -> (Batch_Size, 4, Height / 8, Width / 8)
        x = self.conv(x)
        
        # (Batch_Size, 4, Height / 8, Width / 8) 
        return x
